In [1]:
from data_crawler import *
from loguru import logger
logger.remove()
logger.add("data_crawler.log", format="{time} {level} {message}", level="DEBUG", enqueue=True, backtrace=True)
logger.add("data_crawler_error.log", format="{time} {level} {message}", level="ERROR", enqueue=True, backtrace=True)

2

In [2]:
craw2df = Crawl2DF(4)
craw2df.run()

  0%|          | 0/4537 [00:00<?, ?it/s]


Exception: WTF

In [10]:
from data_provider import *
pd.options.mode.chained_assignment = 'raise'

df = craw2df.dm.df
symbol = "لپيام"
df2 = df.loc[df["symbol"]==symbol].copy()
add_adjust(df2)
print(df2["adj_ending"])
craw2df.dm.get(symbol)
# print(len(df.loc[df.symbol == symbol, "ending"]))
# print(len(df.loc[df.symbol == symbol, "lastday"]))
# df.loc[:, ['endings', 'lastday']]

date
2001-03-25    1594.0
2001-03-26    1594.0
2001-03-27    1594.0
2001-03-28    1589.0
2001-03-31    1586.0
2001-04-03    1559.0
2001-04-07    1579.0
2001-04-08    1559.0
2001-04-09    1546.0
2001-04-10    1590.0
2001-04-14    1587.0
2001-04-15    1560.0
2001-04-16    1570.0
2001-04-17    1570.0
2001-04-21    1565.0
2001-04-22    1560.0
2001-04-23    1560.0
2001-04-24    1589.0
2001-04-25    1590.0
2001-04-28    1600.0
2001-04-29    1680.0
2001-04-30    1669.0
2001-05-01    1650.0
2001-05-02    1599.0
2001-05-05    1593.0
2001-05-06    1591.0
2001-05-07    1576.0
2001-05-08    1570.0
2001-05-09    1566.0
2001-05-12    1569.0
               ...  
2019-08-24    6589.0
2019-08-26    6124.0
2019-08-28    5591.0
2019-08-31    5747.0
2019-09-02    5575.0
2019-09-16    5352.0
2019-09-17    5246.0
2019-09-18    5245.0
2019-09-21    5305.0
2019-09-22    5396.0
2019-09-23    5349.0
2019-09-24    5378.0
2019-09-25    5385.0
2019-09-28    5491.0
2019-09-29    5588.0
2019-09-30    5656.0
2019-10-

TypeError: 'NoneType' object is not iterable

In [ ]:
dm = DataModel("../../xcels", ["master0.csv", "master1.csv"])
dm.read()
dm.store_in_pystore()
dm.restore_from_pystore()

In [4]:
dm = DataModel("../../xcels", [])
dm.restore_from_pystore()
dm.store_in_pystore()

In [5]:
# dm.df[["name", "year", "month", "day"]]

""
